# 2. Data Cleaning and Preprocessing

**Objective:** Clean the raw survey data by handling missing values, standardizing categorical and frequency data, and removing unnecessary columns. The cleaned data will be exported for further analysis.

**Input:** `data/raw/mxmh_survey_results.csv`
**Output:** `data/processed/cleaned_survey_data.csv`

In [4]:
import pandas as pd
import numpy as np
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
# Define paths
RAW_DATA_PATH = os.path.join('..', 'data', 'raw', 'mxmh_survey_results.csv')
PROCESSED_DATA_DIR = os.path.join('..', 'data', 'processed')
CLEANED_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, 'cleaned_survey_data.csv')

# Create processed data directory if it doesn't exist
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Load the raw dataset
try:
    df = pd.read_csv(RAW_DATA_PATH)
    print(f"Raw dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: Raw data file not found at {RAW_DATA_PATH}")
    df = pd.DataFrame() # Avoid errors later

Raw dataset loaded successfully. Shape: (736, 33)


## 2.1 Initial Cleaning: Drop Unnecessary Columns

Drop columns that are not needed for the analysis, such as timestamps or permissions fields.

In [5]:
# Drop Timestamp and Permissions columns
columns_to_drop = ['Timestamp', 'Permissions']
df_cleaned = df.drop(columns=columns_to_drop)
print(f"Dropped columns: {columns_to_drop}. New shape: {df_cleaned.shape}")

Dropped columns: ['Timestamp', 'Permissions']. New shape: (736, 31)


## 2.2 Handle Missing Values

Address missing values based on the analysis from the previous notebook (01_Data_Overview).
- **Age:** Impute with median.
- **Primary streaming service:** Impute with mode.
- **While working:** Impute with mode.
- **Instrumentalist:** Impute with mode.
- **Composer:** Impute with mode.
- **Foreign languages:** Impute with mode.
- **BPM:** Impute with overall median due to high missing count and complexity of genre-based imputation. Note this limitation.
- **Music effects:** Impute with 'No effect'.

In [6]:
# Impute Age with median
age_median = df_cleaned['Age'].median()
df_cleaned['Age'].fillna(age_median, inplace=True)
print(f"Imputed 'Age' with median: {age_median}")

# Impute Primary streaming service with mode
streaming_mode = df_cleaned['Primary streaming service'].mode()[0]
df_cleaned['Primary streaming service'].fillna(streaming_mode, inplace=True)
print(f"Imputed 'Primary streaming service' with mode: {streaming_mode}")

# Impute While working with mode
working_mode = df_cleaned['While working'].mode()[0]
df_cleaned['While working'].fillna(working_mode, inplace=True)
print(f"Imputed 'While working' with mode: {working_mode}")

# Impute Instrumentalist with mode
instrumentalist_mode = df_cleaned['Instrumentalist'].mode()[0]
df_cleaned['Instrumentalist'].fillna(instrumentalist_mode, inplace=True)
print(f"Imputed 'Instrumentalist' with mode: {instrumentalist_mode}")

# Impute Composer with mode
composer_mode = df_cleaned['Composer'].mode()[0]
df_cleaned['Composer'].fillna(composer_mode, inplace=True)
print(f"Imputed 'Composer' with mode: {composer_mode}")

# Impute Foreign languages with mode
foreign_lang_mode = df_cleaned['Foreign languages'].mode()[0]
df_cleaned['Foreign languages'].fillna(foreign_lang_mode, inplace=True)
print(f"Imputed 'Foreign languages' with mode: {foreign_lang_mode}")

# Impute BPM with median
# First, convert BPM to numeric, coercing errors (like non-numeric entries if any)
df_cleaned['BPM'] = pd.to_numeric(df_cleaned['BPM'], errors='coerce')
bpm_median = df_cleaned['BPM'].median()
df_cleaned['BPM'].fillna(bpm_median, inplace=True)
print(f"Imputed 'BPM' with median: {bpm_median}")

# Impute Music effects with 'No effect'
music_effects_fill = 'No effect'
df_cleaned['Music effects'].fillna(music_effects_fill, inplace=True)
print(f"Imputed 'Music effects' with: {music_effects_fill}")

# Verify remaining missing values
print("\nRemaining missing values after imputation:")
print(df_cleaned.isnull().sum().sort_values(ascending=False).head())

Imputed 'Age' with median: 21.0
Imputed 'Primary streaming service' with mode: Spotify
Imputed 'While working' with mode: Yes
Imputed 'Instrumentalist' with mode: No
Imputed 'Composer' with mode: No
Imputed 'Foreign languages' with mode: Yes
Imputed 'BPM' with median: 120.0
Imputed 'Music effects' with: No effect

Remaining missing values after imputation:
Age                 0
Frequency [Jazz]    0
OCD                 0
Insomnia            0
Depression          0
dtype: int64


/tmp/ipykernel_634378/3061550049.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Age'].fillna(age_median, inplace=True)
/tmp/ipykernel_634378/3061550049.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

## 2.3 Clean Categorical Data

Review unique values in key categorical columns to ensure consistency. No major cleaning needed based on initial overview, but good practice to check.

In [7]:
# Example: Check unique values for 'Fav genre'
print("\nUnique values in 'Fav genre':")
print(df_cleaned['Fav genre'].unique())

# Example: Check unique values for 'Primary streaming service'
print("\nUnique values in 'Primary streaming service':")
print(df_cleaned['Primary streaming service'].unique())

# Add checks for other relevant categorical columns if needed


Unique values in 'Fav genre':
['Latin' 'Rock' 'Video game music' 'Jazz' 'R&B' 'K pop' 'Country' 'EDM'
 'Hip hop' 'Pop' 'Rap' 'Classical' 'Metal' 'Folk' 'Lofi' 'Gospel']

Unique values in 'Primary streaming service':
['Spotify' 'Pandora' 'YouTube Music' 'I do not use a streaming service.'
 'Apple Music' 'Other streaming service']


## 2.4 Standardize Frequency Labels

Map the categorical frequency labels ("Never", "Rarely", "Sometimes", "Very frequently") to numerical values (0-3) for easier analysis and modeling.

In [8]:
# Identify frequency columns
freq_cols = [col for col in df_cleaned.columns if col.startswith('Frequency [')]
print(f"\nFound {len(freq_cols)} frequency columns.")

# Define the mapping
freq_mapping = {
    'Never': 0,
    'Rarely': 1,
    'Sometimes': 2,
    'Very frequently': 3
}

# Apply the mapping
for col in freq_cols:
    df_cleaned[col] = df_cleaned[col].map(freq_mapping)
    # Optional: Convert to integer type if desired, handle potential NaNs if mapping failed
    # df_cleaned[col] = df_cleaned[col].astype('Int64') # Use nullable integer type

print("Applied frequency mapping.")
# Display a sample of the transformed columns
display(df_cleaned[freq_cols].head())
# Check data types of frequency columns
print("\nData types of frequency columns after mapping:")
print(df_cleaned[freq_cols].dtypes)


Found 16 frequency columns.
Applied frequency mapping.


,Frequency [Classical],Frequency [Country],Frequency [EDM],Frequency [Folk],Frequency [Gospel],Frequency [Hip hop],Frequency [Jazz],Frequency [K pop],Frequency [Latin],Frequency [Lofi],Frequency [Metal],Frequency [Pop],Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music]
0,1,0,1,0,0,2,0,3,3,1,0,3,2,3,0,2
1,2,0,0,1,2,1,3,1,2,1,0,2,2,1,3,1
2,0,0,3,0,0,1,1,3,0,2,2,1,0,1,1,3
3,2,0,0,1,2,0,3,2,3,2,0,2,2,0,0,0
4,0,0,1,0,1,3,0,3,2,2,0,2,3,3,0,1



Data types of frequency columns after mapping:
Frequency [Classical]           int64
Frequency [Country]             int64
Frequency [EDM]                 int64
Frequency [Folk]                int64
Frequency [Gospel]              int64
Frequency [Hip hop]             int64
Frequency [Jazz]                int64
Frequency [K pop]               int64
Frequency [Latin]               int64
Frequency [Lofi]                int64
Frequency [Metal]               int64
Frequency [Pop]                 int64
Frequency [R&B]                 int64
Frequency [Rap]                 int64
Frequency [Rock]                int64
Frequency [Video game music]    int64
dtype: object


## 2.5 Verify Data Types

Ensure columns have appropriate data types after cleaning. For example, Age and BPM should be numeric. Frequency columns should be numeric (int or float).

In [9]:
print("\nFinal Data Types:")
print(df_cleaned.info())

# Convert Age to integer if it's float after imputation
if pd.api.types.is_float_dtype(df_cleaned['Age']):
     df_cleaned['Age'] = df_cleaned['Age'].astype(int)
     print("\nConverted 'Age' column to integer type.")

# Convert BPM to integer if desired and appropriate (check if decimals are meaningful)
# For now, leave as float due to median imputation potentially introducing decimals
# if pd.api.types.is_float_dtype(df_cleaned['BPM']):
#     df_cleaned['BPM'] = df_cleaned['BPM'].astype(int)
#     print("\nConverted 'BPM' column to integer type.")

# Convert Frequency columns to integer (using nullable Int64 to handle potential NaNs if any step failed)
for col in freq_cols:
    df_cleaned[col] = df_cleaned[col].astype('Int64')
print("\nConverted frequency columns to Int64 type.")

print("\nFinal Data Types after potential conversions:")
print(df_cleaned.info())


Final Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           736 non-null    float64
 1   Primary streaming service     736 non-null    object 
 2   Hours per day                 736 non-null    float64
 3   While working                 736 non-null    object 
 4   Instrumentalist               736 non-null    object 
 5   Composer                      736 non-null    object 
 6   Fav genre                     736 non-null    object 
 7   Exploratory                   736 non-null    object 
 8   Foreign languages             736 non-null    object 
 9   BPM                           736 non-null    float64
 10  Frequency [Classical]         736 non-null    int64  
 11  Frequency [Country]           736 non-null    int64  
 12  Frequency [EDM]               736 non-null   

## 2.6 Export Cleaned Data

Save the cleaned DataFrame to a new CSV file in the `data/processed` directory.

In [10]:
# Export the cleaned DataFrame
try:
    df_cleaned.to_csv(CLEANED_DATA_PATH, index=False)
    print(f"\nCleaned data successfully exported to: {CLEANED_DATA_PATH}")
except Exception as e:
    print(f"\nError exporting data: {e}")

# Display final shape and head
print(f"\nFinal cleaned dataset shape: {df_cleaned.shape}")
print("\nFirst 5 rows of cleaned data:")
display(df_cleaned.head())


Cleaned data successfully exported to: ../data/processed/cleaned_survey_data.csv

Final cleaned dataset shape: (736, 31)

First 5 rows of cleaned data:


,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,BPM,Frequency [Classical],Frequency [Country],Frequency [EDM],Frequency [Folk],Frequency [Gospel],Frequency [Hip hop],Frequency [Jazz],Frequency [K pop],Frequency [Latin],Frequency [Lofi],Frequency [Metal],Frequency [Pop],Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects
0,18,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,156.0,1,0,1,0,0,2,0,3,3,1,0,3,2,3,0,2,3.0,0.0,1.0,0.0,No effect
1,63,Pandora,1.5,Yes,No,No,Rock,Yes,No,119.0,2,0,0,1,2,1,3,1,2,1,0,2,2,1,3,1,7.0,2.0,2.0,1.0,No effect
2,18,Spotify,4.0,No,No,No,Video game music,No,Yes,132.0,0,0,3,0,0,1,1,3,0,2,2,1,0,1,1,3,7.0,7.0,10.0,2.0,No effect
3,61,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,84.0,2,0,0,1,2,0,3,2,3,2,0,2,2,0,0,0,9.0,7.0,3.0,3.0,Improve
4,18,Spotify,4.0,Yes,No,No,R&B,Yes,No,107.0,0,0,1,0,1,3,0,3,2,2,0,2,3,3,0,1,7.0,2.0,5.0,9.0,Improve
